In [0]:
%run "./Configuration"

In [0]:
# Define the metadata widget:
dbutils.widgets.text("entityName", "")
dbutils.widgets.text("entitySourceQuery", "")

# Save the parameter to a variable:
entityName = dbutils.widgets.get('entityName')
url = dbutils.widgets.get('entitySourceQuery')

In [0]:
if entityName in ['customers', 'products']:
    # Get the CSV file content as a string:
    response = requests.get(url)
    
    # Define temp path in DBFS:
    temp_path = f'/dbfs/tmp/{entityName}.csv'
    
    # Write the string to a csv file in DBFS:
    with open(temp_path, 'wb') as f:
        f.write(response.content)
    
    # Create a DataFrame from the CSV file:
    df = spark.read.csv(f'dbfs:/tmp/{entityName}.csv', header=True, inferSchema=True)

In [0]:
# Set folder path to todays date:
folder_path = datetime.datetime.now().strftime("%Y/%m/%d")

In [0]:
# Check data exists:
if df.count() > 0:
    # Set location for parquet:
    location = RAW + f"/AdventureWorks/{entityName}/{folder_path}/"
    
    # Write parquet to data lake:
    df.write.mode('overwrite').format('parquet').option('header', 'true').save(location)

In [0]:
# Output location of parquet as a parameter
dbutils.notebook.exit(file_path)